In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import statsmodels.api as sm
import os
import sys

sys.path.insert(0, os.path.abspath('../developer'))

from config import MOCK_DATA, CODE, OUT
from developer.utilities import read_yaml
from developer.analysis.model import load_model
import re
import plotly.express as px
import plotly.graph_objects as go


In [37]:
df_total = pd.read_csv(OUT / "data" / "data_regression.csv")

##### regression

In [38]:
outcome = 'support'
explanatory_vars = [col for col in df_total.columns if "att" in col] + ['ID']


X = df_total[explanatory_vars].astype(int)
y = df_total[outcome].astype(int)

X = sm.add_constant(X)
model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups':X['ID']})

In [39]:
model.summary()

c:\ProgramData\Anaconda3\envs\conjoint\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 22, but rank is 18
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                support   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     37.54
Date:                Fri, 08 Sep 2023   Prob (F-statistic):          9.04e-105
Time:                        12:56:23   Log-Likelihood:                -9825.2
No. Observations:               15684   AIC:                         1.969e+04
Df Residuals:                   15665   BIC:                         1.983e+04
Df Model:                          18                                         
Covariance Type:              cluster                                         
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         0.4807      0.023     20.767      0.000       0.435       0.526
att_1_Eliminate2050           0.0570      0.006      9.395      0.000       0.045       0.069
att_1_Eliminate2070          -0.0028      0.007     -0.402      0.688      -0.016       0.011
att_1_Reduce2030              0.0747      0.006     12.043      0.000       0.063       0.087
att_2_EnergyAccess            0.0411      0.007      6.213      0.000       0.028       0.054
att_2_InsureWorkers           0.0567      0.007      8.534      0.000       0.044       0.070
att_2_LowPrices               0.0537      0.007      8.006      0.000       0.041       0.067
att_2_NothingSoc             -0.0226      0.007     -3.148      0.002      -0.037      -0.009
att_3_IdentityCoalRegions     0.0282      0.007      4.053      0.000       0.015       0.042
att_3_IndustryCoalRegions     0.0534      0.007      8.086      0.000       0.040       0.066
att_3_NothingEco             -0.0104      0.008     -1.380      0.168      -0.025       0.004
att_3_WorkersCoalRegion       0.0578      0.007      8.623      0.000       0.045       0.071
att_4_Gov&Businesses          0.0140      0.008      1.752      0.080      -0.002       0.030
att_4_Gov&CivilSociety        0.0324      0.008      3.951      0.000       0.016       0.048
att_4_Gov&LaborUnions         0.0197      0.008      2.316      0.021       0.003       0.036
att_4_Gov&LocalGov            0.0381      0.008      4.572      0.000       0.022       0.054
att_4_Gov&Researchers         0.0327      0.008      3.938      0.000       0.016       0.049
att_4_GovAlone               -0.0079      0.009     -0.914      0.360      -0.025       0.009
att_5_Fin&TechSupport         0.0851      0.007     12.806      0.000       0.072       0.098
att_5_FinSupportOnly          0.0213      0.007      3.057      0.002       0.008       0.035
att_5_NoInterference          0.0019      0.007      0.251      0.802      -0.013       0.016
att_5_TechSupportOnly         0.0206      0.007      2.955      0.003       0.007       0.034
ID                         8.795e-05   1.98e-05      4.443      0.000    4.92e-05       0.000
==============================================================================
Omnibus:                    13093.039   Durbin-Watson:                   1.340
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2738.605
Skew:                          -0.836   Prob(JB):                         0.00
Kurtosis:                       1.818   Cond. No.                     6.25e+18
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is 2.29e-28. This might indicate that there are
strong multicolli

In [51]:
# Select the design matrix (explanatory variables) and the outcome variables
outcome = 'utility_standardized'
explanatory_vars = [col for col in df_total.columns if "att" in col]# + ['ID']


X = df_total[explanatory_vars].astype(int)
y = df_total[outcome].astype(int)

X = sm.add_constant(X)
model = sm.OLS(y, X).fit()#cov_type='cluster', cov_kwds={'groups':X['ID']}

In [61]:
import numpy as np
import pandas as pd

df_total = pd.read_csv(OUT / "data" / "data_regression.csv")

outcome = 'support'
explanatory_vars = [col for col in df_total.columns if "att" in col]
explanatory_vars = explanatory_vars[:7] +  ['ID']
#df_total['support'] = df_total['utility'] >= 6

X = df_total[explanatory_vars].astype(int)
X = sm.add_constant(X)

y = df_total[outcome].astype(int)

logit_model = sm.Logit(y, X).fit(cov_type='cluster', cov_kwds={'groups':X['ID']})
#logit_result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.603597
         Iterations 5


In [45]:
outcome = 'utility'
explanatory_vars = [col for col in df_total.columns if "att" in col] + ['ID']

df_control = df_total[df_total['treatment_status'] == 0]

X = df_control[explanatory_vars].astype(int)
y = df_control[outcome].astype(int)

X = sm.add_constant(X)
model_control = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups':X['ID']})

In [5]:
outcome = 'utility'
explanatory_vars = [col for col in df_total.columns if "att" in col] + ['ID']

df_treated = df_total[df_total['treatment_status'] == 1]

X = df_treated[explanatory_vars].astype(int)
y = df_treated[outcome].astype(int)

X = sm.add_constant(X)
model_treated = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups':X['ID']})

In [7]:
model.summary()

c:\ProgramData\Anaconda3\envs\conjoint\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 27, but rank is 8
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                support   R-squared:                       0.197
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.822
Date:                Thu, 31 Aug 2023   Prob (F-statistic):              0.207
Time:                        17:50:03   Log-Likelihood:                -66.507
No. Observations:                 108   AIC:                             177.0
Df Residuals:                      86   BIC:                             236.0
Df Model:                          21                                         
Covariance Type:              cluster                                         
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               0.2570      0.041      6.265      0.000       0.177       0.337
att_1_Eliminate&UseAllOther         0.0009      0.065      0.014      0.989      -0.127       0.129
att_1_Eliminate&UseRenewables      -0.0196      0.102     -0.193      0.847      -0.219       0.179
att_1_Reduce&IncreaseAllOther       0.1865      0.068      2.750      0.006       0.054       0.319
att_1_Reduce&IncreaseRenewables     0.0893      0.116      0.767      0.443      -0.139       0.317
att_2_EnergyAccess                  0.1501      0.127      1.184      0.236      -0.098       0.398
att_2_HealthEdu                     0.1678      0.091      1.838      0.066      -0.011       0.347
att_2_InsureWorkers                -0.0522      0.116     -0.450      0.653      -0.279       0.175
att_2_LowPrices                    -0.1381      0.117     -1.182      0.237      -0.367       0.091
att_2_NothingSoc                    0.1295      0.101      1.286      0.198      -0.068       0.327
att_3_IdentityCoalRegions           0.0063      0.154      0.041      0.967      -0.295       0.308
att_3_IndustryCoalRegions           0.0192      0.073      0.261      0.794      -0.125       0.163
att_3_NothingEco                    0.1063      0.104      1.025      0.305      -0.097       0.309
att_3_WorkersCoalRegion             0.1253      0.086      1.452      0.147      -0.044       0.295
att_4_Gov&Businesses                0.0218      0.111      0.197      0.844      -0.195       0.239
att_4_Gov&CivilSociety              0.1272      0.181      0.702      0.483      -0.228       0.482
att_4_Gov&LaborUnions              -0.0638      0.101     -0.634      0.526      -0.261       0.134
att_4_Gov&LocalGov                  0.0088      0.163      0.054      0.957      -0.311       0.328
att_4_Gov&Researchers               0.4291      0.129      3.334      0.001       0.177       0.681
att_4_GovAlone                     -0.2660      0.111     -2.392      0.017      -0.484      -0.048
att_5_Fin&TechSupport               0.1318      0.125      1.055      0.291      -0.113       0.377
att_5_FinSupportOnly                0.1161      0.099      1.173      0.241      -0.078       0.310
att_5_NoInterference               -0.0370      0.072     -0.514      0.608      -0.178       0.104
att_5_TechSupportOnly               0.0461      0.082      0.563      0.574      -0.114       0.207
att_6_MinorEfforts                  0.0115      0.074      0.154      0.877      -0.134       0.157
att_6_ModerateEfforts               0.1015      0.082      1.242      0.214      -0.059       0.262
att_6_SignificantEffort             0.1440      0.043      3.389      0.001       0.061       0.227
ID                                 -0.0210      0.016     -1.326      0.185      -0

In [5]:
model = load_model(OUT / "models" / "model.pickle")
model.nobs


888.0

In [8]:
print("Regression results for Attribute A:")
print(model.summary())

Regression results for Attribute A:
                            OLS Regression Results                            
Dep. Variable:                utility   R-squared:                       0.198
Model:                            OLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                    0.5149
Date:                Sat, 12 Aug 2023   Prob (F-statistic):              0.845
Time:                        12:53:42   Log-Likelihood:                -137.13
No. Observations:                 132   AIC:                             318.3
Df Residuals:                     110   BIC:                             381.7
Df Model:                          21                                         
Covariance Type:              cluster                                         
                                             coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------

c:\ProgramData\Anaconda3\envs\conjoint\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 26, but rank is 10
  warnings.warn('covariance of constraints does not have full '


## Descriptives


In [2]:
clean_data = pd.read_csv(OUT / "data" / "data_clean.csv")

import pandas as pd
import plotly.graph_objs as go

def attribute_support(df, attribute):
    df = df.copy()

    df = df[[attribute, 'support']]
    df['support'] = df['support'].astype(int)

    categories = df[attribute].unique()

    support = {"Attribute Level": [], "Value": [], "CI_lower": [], "CI_upper": []}
    
    for cat in categories[::-1]:
        group = df[df[attribute] == cat]
        mean = group['support'].mean()
        std_dev = group['support'].std()
        n = len(group)
        confidence_interval = 1.96 * (std_dev / (n**0.5))  # 95% confidence interval

        support["Attribute Level"].append(cat.replace('&', '<br>'))
        support["Value"].append(mean.round(2))
        support["CI_lower"].append((mean - confidence_interval).round(2))
        support["CI_upper"].append((mean + confidence_interval).round(2))

    df = pd.DataFrame(support)

    color_scale = ["rgb(173, 221, 142)", "rgb(127, 188, 65)", "rgb(78, 139, 37)", "rgb(45, 82, 21)"]

    fig = go.Figure()

    for i, row in df.iterrows():
        fig.add_trace(go.Bar(
            x=[row["Attribute Level"]],
            y=[row["Value"]],
            error_y=dict(
                type='data',
                array=[row["CI_upper"] - row["Value"]],
                arrayminus=[row["Value"] - row["CI_lower"]],
                visible=True
            ),
            marker_color=color_scale[i],
            name=row["Attribute Level"]
        ))

    # Set y-axis range from 0 to 1
    fig.update_layout(yaxis_range=[0, 1], width=600, height=500)

    # Add a horizontal line at y=0.5
    
    fig.add_hline(y=0.5, line_dash="dash")


    fig.update_layout(barmode="group", bargap=0.6, bargroupgap=0.1)
    fig.update_layout(
        title={
            'text': "Fig 1: Support of the different phase-out strategies",
            'x': 0.5,
            'xanchor': 'center',
            'font': {'family': 'Computer Modern'}
        },
        margin=dict(l=20, r=20, t=45, b=5),
        paper_bgcolor="#EADDCA",
        plot_bgcolor='rgba(0,0,0,0)',
        showlegend=False,  # Show legend for different Attribute Levels
        xaxis_showticklabels=True,
        xaxis_title=None,
    )

    return df



fig = attribute_support(clean_data, "att_1")

#fig.show()




In [3]:
fig

,Attribute Level,Value,CI_lower,CI_upper
0,Reduce<br>IncreaseRenewables,0.70,0.66,0.75
1,Eliminate<br>UseAllOther,0.76,0.72,0.81
2,Reduce<br>IncreaseAllOther,0.72,0.68,0.77
3,Eliminate<br>UseRenewables,0.77,0.73,0.82


## Plots

##### 1.1 Relative support plot AMCE (Intra)

In [40]:
model.params[0]['att_6_SignificantEffort']

0.3932151622585344

In [13]:
import plotly.graph_objects as go

list1 = read_yaml(r"C:\Users\sjurl\OneDrive\Desktop\MasterThesis\Analysis\conjoint\developer\final\plot_specs.yaml")


# Assuming 'model' is the variable that holds your regression results
# Extract the coefficients and standard errors for each attribute level
att_1_levels = list1['order']['att_1']
att_2_levels = list1['order']['att_2']
att_3_levels = list1['order']['att_3']
att_4_levels = list1['order']['att_4']
att_5_levels = list1['order']['att_5']
att_6_levels = list1['order']['att_6']



#Remember to add att_6
att_levels = [att_6_levels, att_5_levels, att_4_levels, att_3_levels, att_2_levels, att_1_levels]

att_colors = ['gray', 'red', 'blue', 'green', 'orange', 'purple']  # Colors for each attribute group

fig = go.Figure()

total_levels = sum(len(levels) for levels in att_levels)

# Loop through each attribute group and add the data to the plot
for i, levels in enumerate(att_levels):
    att_coefficients = [model.params[f'att_{6-i}_{level}'] for level in levels]
    att_standard_errors = [model.bse[f'att_{6-i}_{level}'] for level in levels]

    fig.add_trace(go.Scatter(
        x=att_coefficients,
        y=levels,
        mode='markers',
        error_x=dict(type='data', array=att_standard_errors, color=att_colors[i], thickness=1.5),
        marker=dict(color='darkgray', size=10),
        orientation='h',
        showlegend = False,
    ))

    fig.add_shape(
        type="rect",
        x0=-1.5,  # Set a fixed value for x0, which is left side of the plot
        x1=1.5,  # Set the width of the shape to 1000 (right side of the plot)
        y0=total_levels - sum(len(l) for l in att_levels[i:]),  # Set y0 to the starting level index
        y1=total_levels - sum(len(l) for l in att_levels[i:]) + len(levels) - 1,  # Set y1 to the ending level index
        fillcolor=att_colors[i],
        opacity=0.1,  # Set the opacity for a light transparent effect
        layer="below",  # Place the rectangle below the scatter plot markers
    )

# Add a vertical line at x=0 for reference
fig.add_shape(type="line", x0=0, x1=0, y0=att_6_levels[0], y1=att_1_levels[-1], line=dict(color="gray", width=1, dash='dash'))

# Update the layout of the error bar plot
fig.update_layout(
    title='Relative Rating Differences',
    xaxis_title='',
    yaxis_title='Attribute Levels',
    yaxis=dict(categoryorder='array', categoryarray=att_6_levels),  # Set the categoryorder for y-axis based on att_1_levels
    xaxis=dict(tickformat='.2f', zeroline=False),  # Remove x-axis zeroline
    showlegend=True,  # Show legend with attribute names
    margin=dict(l=80, r=30, b=40, t=80),
    height=600,  # Set the height of the plot to 600 pixels
    width=1000,
    title_x=0.62,
)

# Show the interactive error bar plot
fig.show()


Grouped:

In [10]:
import plotly.graph_objects as go

# Assuming 'model' is the variable that holds your regression results
# Extract the coefficients and standard errors for each attribute level
att_1_levels = ['PhaseOut', 'Stop&Reduce', 'Stop&Maintain', 'StatusQuo']
att_2_levels = ['HighInvestment&Int', 'HighInvestment&Int&Consideration', 'LowInvestment&LowConsideration', 'LowInvestment']
att_3_levels = ['HealthEdu', 'EnergyAccess', 'LowPrices', 'Transfers', 'NothingSoc']
att_4_levels = ['CreateJobs', 'EarlyPension', 'JobGuarantee', 'Retrain', 'NothingEco']
att_5_levels = ['CivilNGO', 'EnergySector', 'LabourUnion', 'LocalGov', 'Media', 'Researchers', 'CentralGov']



#Remember to add att_6
att_levels = [att_5_levels, att_4_levels, att_3_levels, att_2_levels, att_1_levels]

att_colors_control = ['red', 'blue', 'green', 'orange', 'purple']  # Colors for each attribute group

fig = go.Figure()

total_levels = sum(len(levels) for levels in att_levels)

# Loop through each attribute group and add the data to the plot
fig = go.Figure()

# Loop through each attribute group and add the data for 'control' to the plot
for i, levels in enumerate(att_levels):
    att_coefficients = [model_control.params[f'att_{5-i}_{level}'] for level in levels]
    att_standard_errors = [model_control.bse[f'att_{5-i}_{level}'] for level in levels]

    relative_differences = [coeff - att_coefficients[-1] for coeff in att_coefficients]

    fig.add_trace(go.Scatter(
        x=relative_differences,
        y=levels,
        mode='markers',
        error_x=dict(type='data', array=att_standard_errors, color=att_colors[i], thickness=1.5),
        marker=dict(color='darkgray', size=10),
        orientation='h',
        showlegend=False,
        name='Control',  # Add a legend name for the control group
    ))

# Loop through each attribute group and add the data for 'treated' to the plot
for i, levels in enumerate(att_levels):
    att_coefficients = [model_treated.params[f'att_{5-i}_{level}'] for level in levels]
    att_standard_errors = [model_treated.bse[f'att_{5-i}_{level}'] for level in levels]

    relative_differences = [coeff - att_coefficients[-1] for coeff in att_coefficients]

    fig.add_trace(go.Scatter(
        x=relative_differences,
        y=levels,
        mode='markers',
        error_x=dict(type='data', array=att_standard_errors, color=att_colors[i], thickness=1.5),
        marker=dict(color=att_colors[i], size=10),  # Use different colors for treated group
        orientation='h',
        showlegend=False,
        name='Treated',  # Add a legend name for the treated group
    ))

    fig.add_shape(
        type="rect",
        x0=-1.5,  # Set a fixed value for x0, which is left side of the plot
        x1=1.5,  # Set the width of the shape to 1000 (right side of the plot)
        y0=total_levels - sum(len(l) for l in att_levels[i:]),  # Set y0 to the starting level index
        y1=total_levels - sum(len(l) for l in att_levels[i:]) + len(levels) - 1,  # Set y1 to the ending level index
        fillcolor=att_colors[i],
        opacity=0.1,  # Set the opacity for a light transparent effect
        layer="below",  # Place the rectangle below the scatter plot markers
    )

# Add a vertical line at x=0 for reference
fig.add_shape(type="line", x0=0, x1=0, y0=att_5_levels[0], y1=att_1_levels[-1], line=dict(color="gray", width=1, dash='dash'))

# Update the layout of the error bar plot
fig.update_layout(
    title='Relative Rating Differences',
    xaxis_title='',
    yaxis_title='Attribute Levels',
    yaxis=dict(categoryorder='array', categoryarray=att_5_levels),  # Set the categoryorder for y-axis based on att_1_levels
    xaxis=dict(tickformat='.2f', zeroline=False),  # Remove x-axis zeroline
    showlegend=True,  # Show legend with attribute names
    margin=dict(l=80, r=30, b=40, t=80),
    height=600,  # Set the height of the plot to 600 pixels
    width=1000,
    title_x=0.62,
)

# Show the interactive error bar plot
fig.show()


In [15]:
def plot_relative_differences_grouped(model_control, model_treated, data_info, group1, group2, width=1.0, plot_title="Marginal Means Treated / Control"):

    nobs_light = model_control.nobs / 12
    nobs_dark = model_treated.nobs / 12

    order = data_info['order']
    att_1_levels = order['att_1']
    att_2_levels = order['att_2']
    att_3_levels = order['att_3']
    att_4_levels = order['att_4']
    att_5_levels = order['att_5']
    att_6_levels = order['att_6']

    att_levels = [att_6_levels, att_5_levels, att_4_levels, att_3_levels, att_2_levels, att_1_levels]

    att_colors_control = data_info['colors_control']
    att_colors_treated = data_info['colors_treated']

    fig = go.Figure()

    total_levels = sum(len(levels) for levels in att_levels)

    # Loop through each attribute group and add the data for 'control' to the plot
    for i, levels in enumerate(att_levels):

        att_coefficients = [model_treated.params[f'att_{6-i}_{level}'] for level in levels]
        att_standard_errors = [model_treated.bse[f'att_{6-i}_{level}'] for level in levels]

        fig.add_trace(go.Scatter(
            x=att_coefficients,
            y=levels,
            mode='markers',
            error_x=dict(type='data', array=att_standard_errors, color=att_colors_treated[i], thickness=1.5),
            marker=dict(color=att_colors_treated[i], size=10),  # Use different colors for treated group
            orientation='h',
            showlegend=False,
            name='Treated',  # Add a legend name for the treated group
        ))
        

    # Loop through each attribute group and add the data for 'treated' to the plot
    for i, levels in enumerate(att_levels):

        att_coefficients = [model_control.params[f'att_{6-i}_{level}'] for level in levels]
        att_standard_errors = [model_control.bse[f'att_{6-i}_{level}'] for level in levels]

        fig.add_trace(go.Scatter(
            x=att_coefficients,
            y=levels,
            mode='markers',
            error_x=dict(type='data', array=att_standard_errors, color=att_colors_control[i], thickness=1.5),
            marker=dict(color=att_colors_control[i], size=10),
            orientation='h',
            showlegend=False,
            name='Control',  # Add a legend name for the control group
        ))

        fig.add_shape(
            type="rect",
            x0=-1.5,  # Set a fixed value for x0, which is left side of the plot
            x1=1.5,  # Set the width of the shape to 1000 (right side of the plot)
            y0=total_levels - sum(len(l) for l in att_levels[i:]),  # Set y0 to the starting level index
            y1=total_levels - sum(len(l) for l in att_levels[i:]) + len(levels) - 1,  # Set y1 to the ending level index
            fillcolor=att_colors_treated[i],
            opacity=0.1,  # Set the opacity for a light transparent effect
            layer="below",  # Place the rectangle below the scatter plot markers
        )

    # Add a vertical line at x=0 for reference
    fig.add_shape(type="line", x0=0, x1=0, y0=att_6_levels[0], y1=att_1_levels[-1], line=dict(color="gray", width=1, dash='dash'))

    fig.add_annotation(
                x=-0.02,  # X-coordinate for the annotation (adjust as needed)
                y=25.0,  # Y-coordinate for the annotation (above the plot)
                text=f"{group1} (lighter): n={(int(nobs_light))}",
                showarrow=False,
                font=dict(
                    family='Computer Modern',
                    size=12,
                    ),
            )
    fig.add_annotation(
                x=-0.02,  # X-coordinate for the annotation (adjust as needed)
                y=26.0,  # Y-coordinate for the annotation (above the plot)
                text=f"{group2} (darker): n={(int(nobs_dark))}",
                showarrow=False,
                font=dict(
                    family='Computer Modern',
                    size=12,
                    ),
            )

    # Update the layout of the error bar plot
    fig.update_layout(
        title={
            'text': plot_title,
            'x': 0.5,
            'font': {'family': 'Computer Modern'}
        },
        xaxis_title='',
        yaxis_title='Attribute Levels',
        yaxis=dict(categoryorder='array', categoryarray=att_6_levels),  # Set the categoryorder for y-axis based on att_1_levels
        xaxis=dict(tickformat='.2f', zeroline=False, range=[-0.15,0.2]),  # Remove x-axis zeroline
        showlegend=True,  # Show legend with attribute names
        margin=dict(l=80, r=30, b=40, t=80),
        height=600,  # Set the height of the plot to 600 pixels
        width=1000,
        paper_bgcolor="#EADDCA",
        plot_bgcolor='rgba(0,0,0,0)',
    )

    # Show the interactive error bar plot
    return fig

specs = read_yaml(r"C:\Users\sjurl\OneDrive\Desktop\MasterThesis\Analysis\conjoint\developer\final\plot_specs.yaml")

model_control = load_model(OUT / "models" / "model_non_coal_region.pickle")
model_treated = load_model(OUT / "models" / "model_coal_region.pickle")



fig = plot_relative_differences_grouped(model_control, model_treated, specs, group1="NonCoal", group2="Coal", width=1.0, plot_title="Marginal Means Coal / NonCoal region")
fig.show()

1.2 Marginal Means

In [11]:
import plotly.graph_objects as go

# Assuming 'model' is the variable that holds your regression results
# Extract the coefficients and standard errors for each attribute level
att_1_levels = ['PhaseOut', 'Stop&Reduce', 'Stop&Maintain', 'StatusQuo']
att_2_levels = ['HighInvestment&Int', 'HighInvestment&Int&Consideration', 'LowInvestment&LowConsideration', 'LowInvestment']
att_3_levels = ['HealthEdu', 'EnergyAccess', 'LowPrices', 'Transfers', 'NothingSoc']
att_4_levels = ['CreateJobs', 'EarlyPension', 'JobGuarantee', 'Retrain', 'NothingEco']
att_5_levels = ['CivilNGO', 'EnergySector', 'LabourUnion', 'LocalGov', 'Media', 'Researchers', 'CentralGov']



#Remember to add att_6
att_levels = [att_5_levels, att_4_levels, att_3_levels, att_2_levels, att_1_levels]

att_colors = ['red', 'blue', 'green', 'orange', 'purple']  # Colors for each attribute group

fig = go.Figure()

total_levels = sum(len(levels) for levels in att_levels)

# Loop through each attribute group and add the data to the plot
for i, levels in enumerate(att_levels):
    att_coefficients = [model.params[f'att_{5-i}_{level}'] for level in levels]
    att_standard_errors = [model.bse[f'att_{5-i}_{level}'] for level in levels]

    fig.add_trace(go.Scatter(
        x=att_coefficients,
        y=levels,
        mode='markers',
        error_x=dict(type='data', array=att_standard_errors, color=att_colors[i], thickness=1.5),
        marker=dict(color='darkgray', size=10),
        orientation='h',
        showlegend = False,
    ))

    fig.add_shape(
        type="rect",
        x0=-1.5,  # Set a fixed value for x0, which is left side of the plot
        x1=1.5,  # Set the width of the shape to 1000 (right side of the plot)
        y0=total_levels - sum(len(l) for l in att_levels[i:]),  # Set y0 to the starting level index
        y1=total_levels - sum(len(l) for l in att_levels[i:]) + len(levels) - 1,  # Set y1 to the ending level index
        fillcolor=att_colors[i],
        opacity=0.1,  # Set the opacity for a light transparent effect
        layer="below",  # Place the rectangle below the scatter plot markers
    )

# Add a vertical line at x=0 for reference
fig.add_shape(type="line", x0=0, x1=0, y0=att_5_levels[0], y1=att_1_levels[-1], line=dict(color="gray", width=1, dash='dash'))

# Update the layout of the error bar plot
fig.update_layout(
    title='Marginal Means',
    xaxis_title='',
    yaxis_title='Attribute Levels',
    yaxis=dict(categoryorder='array', categoryarray=att_5_levels),  # Set the categoryorder for y-axis based on att_1_levels
    xaxis=dict(tickformat='.2f', zeroline=False),  # Remove x-axis zeroline
    showlegend=True,  # Show legend with attribute names
    margin=dict(l=80, r=30, b=40, t=80),
    height=600,  # Set the height of the plot to 600 pixels
    width=1000,
    title_x=0.62,
)

# Show the interactive error bar plot
fig.show()


Grouped

In [12]:
import plotly.graph_objects as go

# Assuming 'model' is the variable that holds your regression results
# Extract the coefficients and standard errors for each attribute level
att_1_levels = ['PhaseOut', 'Stop&Reduce', 'Stop&Maintain', 'StatusQuo']
att_2_levels = ['HighInvestment&Int', 'HighInvestment&Int&Consideration', 'LowInvestment&LowConsideration', 'LowInvestment']
att_3_levels = ['HealthEdu', 'EnergyAccess', 'LowPrices', 'Transfers', 'NothingSoc']
att_4_levels = ['CreateJobs', 'EarlyPension', 'JobGuarantee', 'Retrain', 'NothingEco']
att_5_levels = ['CivilNGO', 'EnergySector', 'LabourUnion', 'LocalGov', 'Media', 'Researchers', 'CentralGov']



#Remember to add att_6
att_levels = [att_5_levels, att_4_levels, att_3_levels, att_2_levels, att_1_levels]

att_colors_control = ['red', 'blue', 'green', 'orange', 'purple']  # Colors for each attribute group

fig = go.Figure()

total_levels = sum(len(levels) for levels in att_levels)

# Loop through each attribute group and add the data to the plot
fig = go.Figure()

# Loop through each attribute group and add the data for 'control' to the plot
for i, levels in enumerate(att_levels):
    att_coefficients = [model_control.params[f'att_{5-i}_{level}'] for level in levels]
    att_standard_errors = [model_control.bse[f'att_{5-i}_{level}'] for level in levels]


    fig.add_trace(go.Scatter(
        x=att_coefficients,
        y=levels,
        mode='markers',
        error_x=dict(type='data', array=att_standard_errors, color=att_colors[i], thickness=1.5),
        marker=dict(color='darkgray', size=10),
        orientation='h',
        showlegend=False,
        name='Control',  # Add a legend name for the control group
    ))

# Loop through each attribute group and add the data for 'treated' to the plot
for i, levels in enumerate(att_levels):
    att_coefficients = [model_treated.params[f'att_{5-i}_{level}'] for level in levels]
    att_standard_errors = [model_treated.bse[f'att_{5-i}_{level}'] for level in levels]

    fig.add_trace(go.Scatter(
        x=att_coefficients,
        y=levels,
        mode='markers',
        error_x=dict(type='data', array=att_standard_errors, color=att_colors[i], thickness=1.5),
        marker=dict(color=att_colors[i], size=10),  # Use different colors for treated group
        orientation='h',
        showlegend=False,
        name='Treated',  # Add a legend name for the treated group
    ))

    fig.add_shape(
        type="rect",
        x0=-1.5,  # Set a fixed value for x0, which is left side of the plot
        x1=1.5,  # Set the width of the shape to 1000 (right side of the plot)
        y0=total_levels - sum(len(l) for l in att_levels[i:]),  # Set y0 to the starting level index
        y1=total_levels - sum(len(l) for l in att_levels[i:]) + len(levels) - 1,  # Set y1 to the ending level index
        fillcolor=att_colors[i],
        opacity=0.1,  # Set the opacity for a light transparent effect
        layer="below",  # Place the rectangle below the scatter plot markers
    )

# Add a vertical line at x=0 for reference
fig.add_shape(type="line", x0=0, x1=0, y0=att_5_levels[0], y1=att_1_levels[-1], line=dict(color="gray", width=1, dash='dash'))

# Update the layout of the error bar plot
fig.update_layout(
    title='Marginal Means Treatment/Control',
    xaxis_title='',
    yaxis_title='Attribute Levels',
    yaxis=dict(categoryorder='array', categoryarray=att_5_levels),  # Set the categoryorder for y-axis based on att_1_levels
    xaxis=dict(tickformat='.2f', zeroline=False),  # Remove x-axis zeroline
    showlegend=True,  # Show legend with attribute names
    margin=dict(l=80, r=30, b=40, t=80),
    height=600,  # Set the height of the plot to 600 pixels
    width=1000,
    title_x=0.62,
)

# Show the interactive error bar plot
fig.show()


##### 1.2 Rating attributes with a Normalization Method.

*Normalizing the coefficients involves transforming them to a common scale, typically between 0 and 1. This makes it easier to compare the relative importance of different attributes.*


To aggregate the importance scores for different levels of the same attribute and obtain an overall importance score for each attribute, you can calculate a weighted average or sum of the importance scores of its individual levels. Here's how you can do it:

Calculate Normalized Importance Scores (NIS) for Attribute Levels:

Follow the normalization method as described earlier to calculate the normalized importance scores (NIS) for each attribute level.
Aggregate Importance Scores for Each Attribute:

a. Weighted Average Method:

Calculate the weighted average importance score for each attribute by taking the sum of the products of each level's NIS and its corresponding weight (frequency or probability of that level's occurrence in the choice sets).
This method considers both the relative importance of each level and its likelihood of being chosen in the experiment.
b. Simple Sum Method:

Sum up the normalized importance scores (NIS) of all levels within an attribute.
This method treats all levels equally in terms of their contribution to the overall importance score.
Attribute Importance Ranking:

Rank the attributes based on the aggregated importance scores. Higher scores indicate greater importance.
Example using the Weighted Average Method:

Let's consider an example with attribute "att_1" from your regression results. You have four levels: "PhaseOut," "StatusQuo," "Stop&Maintain," and "Stop&Reduce." You've already calculated the normalized importance scores (NIS) for each level as follows:

NIS(PhaseOut) = 0.8810 / 0.8810 = 1.0000
NIS(StatusQuo) = 0.2979 / 0.8810 = 0.3379
NIS(Stop&Maintain) = 0.1471 / 0.8810 = 0.1668
NIS(Stop&Reduce) = 0.9077 / 0.8810 = 1.0302
Let's assume that the frequency (or probability) of each level's occurrence in the choice sets is as follows:

PhaseOut: 30%
StatusQuo: 20%
Stop&Maintain: 25%
Stop&Reduce: 25%
Now, calculate the weighted average importance score for "att_1":
Weighted Average Importance Score for att_1 = (NIS(PhaseOut) * 0.30) + (NIS(StatusQuo) * 0.20) + (NIS(Stop&Maintain) * 0.25) + (NIS(Stop&Reduce) * 0.25)
Weighted Average Importance Score for att_1 = (1.0000 * 0.30) + (0.3379 * 0.20) + (0.1668 * 0.25) + (1.0302 * 0.25) ≈ 0.5623

Repeat this process for each attribute to obtain aggregated importance scores, and then rank the attributes based on these scores.

Remember that the choice of the weighting scheme (equal weights, frequency-based weights, or other relevant weights) depends on your specific context and research design.

##### Plot freq

In [25]:
freq = pd.read_csv(OUT / 'data' / 'data_freq.csv')

NIS

##### 1.3 MarginalMeans

In [13]:
df_total = pd.read_csv(OUT / "models" / "model_aware.csv")
df_total.iloc[2,1]


IndexError: index 3 is out of bounds for axis 0 with size 3

In [4]:
df_total = pd.read_csv(OUT / "data" / "data_regression.csv")

import pandas as pd
import numpy as np

def _calculate_conditional_probability(df, column_x, column_y):

    nobs = len(df)
    # Step 1: Count occurrences of X=1 and Y=1 simultaneously
    xy_count = ((df[column_x] == True) & (df[column_y] == True)).sum()
    
    # Step 2: Count occurrences of X=1
    x_count = (df[column_x] == True).sum()
    
    # Step 3: Calculate P(Y=1|X=1)
    if x_count > 0:
        probability_y_given_x = xy_count / x_count
        probability_y_given_x = probability_y_given_x.round(4)
    else:
        probability_y_given_x = np.nan
    
    # Step 4: Calculate standard deviation
    variance_y_given_x = (probability_y_given_x * (1 - probability_y_given_x)) / x_count
    std_deviation = np.sqrt(variance_y_given_x).round(4)
    
    return probability_y_given_x, std_deviation, nobs

def marginal_means(df): 
    attributes_levels = df.columns[df.columns.str.startswith('att')]

    outcome = 'support'

    marginal_means ={}

    for att_level in attributes_levels:
        results = _calculate_conditional_probability(df, att_level, outcome)
        marginal_means[f'{att_level}_MM'] = []
        marginal_means[f'{att_level}_MM'].append(results[0])
        marginal_means[f'{att_level}_MM'].append(results[1])
        marginal_means[f'{att_level}_MM'].append(results[2])

    return pd.DataFrame(marginal_means)

marginal_means = marginal_means(df_total)




In [34]:
data_info = read_yaml(r"C:\Users\sjurl\OneDrive\Desktop\MasterThesis\Analysis\conjoint\developer\final\plot_specs.yaml")


def plot_MM(MM_data, data_info, width=1.0, plot_title="Fig 3: Marginal Means on support for policy attributes"):

    order = data_info['order']
    att_1_levels = order['att_1']
    att_2_levels = order['att_2']
    att_3_levels = order['att_3']
    att_4_levels = order['att_4']
    att_5_levels = order['att_5']

    att_levels = [att_5_levels, att_4_levels, att_3_levels, att_2_levels, att_1_levels]

    att_colors = data_info['colors']

    fig = go.Figure()

    total_levels = sum(len(levels) for levels in att_levels)

    # Loop through each attribute group and add the data to the plot
    for i, levels in enumerate(att_levels):
        att_coefficients = [MM_data.iloc[0][f'att_{5-i}_{level}_MM'] for level in levels]
        att_standard_errors = [MM_data.iloc[1][f'att_{5-i}_{level}_MM']*1.97 for level in levels]

        fig.add_trace(go.Scatter(
            x=att_coefficients,
            y=levels,
            mode='markers',
            error_x=dict(type='data', array=att_standard_errors, color=att_colors[i], thickness=1.5),
            marker=dict(color='#36454F', size=10),
            orientation='h',
            showlegend = False,
        ))

        fig.add_shape(
            type="rect",
            x0=-width,  # Set a fixed value for x0, which is left side of the plot
            x1=width,  # Set the width of the shape to 1000 (right side of the plot)
            y0=total_levels - sum(len(l) for l in att_levels[i:]),  # Set y0 to the starting level index
            y1=total_levels - sum(len(l) for l in att_levels[i:]) + len(levels) - 1,  # Set y1 to the ending level index
            fillcolor=att_colors[i],
            opacity=0.1,  # Set the opacity for a light transparent effect
            layer="below",  # Place the rectangle below the scatter plot markers
        )

    # Add a vertical line at x=0 for reference
    fig.add_shape(type="line", x0=0.5, x1=0.5, y0=att_5_levels[0], y1=att_1_levels[-1], line=dict(color="gray", width=1, dash='dash'))

    # Update the layout of the error bar plot
    fig.update_layout(
        title={
            'text': plot_title,
            'x': 0.0,
            'xanchor': 'center',
            'font': {'family': 'Computer Modern'}
        },
        xaxis_title='AMCE on support (0-1)',
        yaxis_title='Attribute Levels',
        yaxis=dict(categoryorder='array', categoryarray=att_5_levels),  # Set the categoryorder for y-axis based on att_1_levels
        xaxis=dict(tickformat='.2f', zeroline=False, range=[0,1.0]),  # Remove x-axis zeroline
        showlegend=True,  # Show legend with attribute names
        margin=dict(l=80, r=30, b=40, t=80),
        height=800,  # Set the height of the plot to 600 pixels
        width=1000,
        title_x=0.50,
        paper_bgcolor="#EADDCA",
        plot_bgcolor='rgba(0,0,0,0)',
    ) 

    # Show the interactive error bar plot
    return fig

plot_MM(marginal_means, data_info)

##### 1.4 Maybe: PCA Analysis:

Data Preparation:

Prepare your DCE data matrix, where each row represents a respondent's choice set, and columns represent different attribute levels.
Standardization:

Standardize the data by subtracting the mean and dividing by the standard deviation for each attribute. This ensures that all attributes are on similar scales and prevents attributes with larger variances from dominating the PCA.
Perform PCA:

Apply PCA to the standardized data matrix. The output of PCA will include the principal components and their associated eigenvalues.
Interpretation:

Examine the explained variance for each principal component. This helps you understand how much of the total variance in the data each component explains.
Look at the loadings (weights) of the original attributes on each principal component. These loadings indicate the strength and direction of the relationship between the attribute and the principal component.
Attribute Relationships:

PCA can provide insights into how attributes are related to each other. For example, attributes that have high loadings on the same principal component are positively correlated, while those with opposite loadings are negatively correlated.
Decision Support:

While PCA itself may not directly provide attribute importance scores for policy package choices, the derived principal components can help you identify patterns or relationships that might influence choices. These insights can then be used in conjunction with other methods to understand attribute importance.